In [ ]:
# Import modules
## Point cloud and calculation modules
import laspy as lp
import numpy as np
from numpy import arange
import pandas as pd
import csv
import math
import numba as nb

## Visualization modules
import open3d as o3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Fitting modules
from scipy.optimize import curve_fit

## Time calculation modules
import time
import datetime

In [ ]:
# Import data from *.txt file function
def data_import(or_x, or_y, or_z, path):
    
    # ------------------------------------------------------------------------------- #
    # Input parameters
    #      - or_x, or_y, or_z : station position in x, y, and z coordinate (float)
    #      - path             : *.txt file location                        (string)
    
    # Output parameters
    #      - staPoint         : x, y, z coordinate of station position     (array)
    #      - inPoint          : point cloud data                           (array)
    #      - coorPoint        : x, y, z coordinate of point cloud data     (array)
    #      - int_arr          : point cloud raw intensity data             (array)
    #      - color            : r, g, b color of point cloud data          (array)
    #      - reflect          : point cloud reflectance data               (array)
    #      - normal           : normal vector in x, y, z direction         (array)
    # ------------------------------------------------------------------------------- #
        
    # Import data origin
    staPoint = np.array([or_x, or_y, or_z])
    staPoint = np.transpose(staPoint).reshape(1,3)

    # Import point cloud
    inPoint=np.loadtxt(path)
    inPoint=np.transpose(inPoint)

    # Select point coordinates from *.las data
    coorPoint = np.array([inPoint[0], inPoint[1], inPoint[2]])
    coorPoint = np.transpose(coorPoint)

    # Select intensity data
    int_arr = np.array(inPoint[12])
    int_arr = np.transpose(int_arr)

    # Select color
    color = np.array([inPoint[3], inPoint[4], inPoint[5]])
    color = np.transpose(color)

    # Select reflectance
    reflect = np.array(inPoint[6])
    reflect = np.transpose(reflect)
    
    # Select normal vector
    normal = np.array([inPoint[13], inPoint[14], inPoint[15]])
    normal = np.transpose(normal)
    
    return staPoint, inPoint, coorPoint, int_arr, color, reflect, normal

In [ ]:
# Distance calculation function
def euc_distance(station, coordinate):

    # ------------------------------------------------------------------------------- #
    # Input parameters
    #         - station      : x, y, z coordinate of station position    (array)
    #         - coordinate   : x, y, z coordinate of point cloud data    (array)
    
    # Output parameters
    #         - distance     : distance between TLS and objects          (array)
    # ------------------------------------------------------------------------------- #
    
    return np.array([[ np.linalg.norm(i-j) for j in coordinate] for i in station])

In [ ]:
# Incidence angle function
def angle_calc(normal, point_cloud, sta_point):
    
    # ------------------------------------------------------------------------------------- #
    # Input parameters
    #         - normal       : normal vector in x, y, z direction         (array)
    #         - point_cloud  : x, y, z coordinate of point cloud data     (array)
    #         - sta_point    : x, y, z coordinate of station position     (array)

    # Output parameters
    #         - angle        : incidence angle of each points             (array/radians)
    #         - cos_ang      : cosine incidence angle                     (array)
    # ------------------------------------------------------------------------------------- #
    
    angle = np.array([np.arccos(np.dot((sta_point-point_cloud[i]), normal[i]) / (np.linalg.norm(sta_point-point_cloud[i]) * np.linalg.norm(normal[i]))) for i in range (0, point_cloud[:,0].size)])

    angle = np.transpose(angle)
    angle = np.where((angle <= np.pi/2), angle, (np.pi - angle))
    cos_ang = np.cos(angle)
    return angle, cos_ang

In [ ]:
def voxel_center_overlap(pt_cloud1,pt_cloud2, intensity1, intensity2, reflectance1, reflectance2, distance1, distance2, angle1, angle2, v_size):
    # Define bottom and upper corner of the point cloud
    p_min = np.array([min(np.append(pt_cloud1[:,0], pt_cloud2[:,0])), min(np.append(pt_cloud1[:,1], pt_cloud2[:,1])), min(np.append(pt_cloud1[:,2], pt_cloud2[:,2]))]) # bottom corner
    p_max = np.array([max(np.append(pt_cloud1[:,0], pt_cloud2[:,0])), max(np.append(pt_cloud1[:,1], pt_cloud2[:,1])), max(np.append(pt_cloud1[:,2], pt_cloud2[:,2]))]) # upper corner
    p_mid = p_min+((p_max-p_min)/2)                                       # middle point
    
    # Calculate maximum voxel number
    max_x_index = math.ceil((p_max[0]-p_min[0]+0.5*v_size)/v_size) # max number in x
    max_y_index = math.ceil((p_max[1]-p_min[1]+0.5*v_size)/v_size) # max number in y
    max_z_index = math.ceil((p_max[2]-p_min[2]+0.5*v_size)/v_size) # max number in z
    
    max_xy_index = max_x_index * max_y_index                       # max number in 2D (x and y)
    max_xyz_index = max_x_index * max_y_index * max_z_index        # max number in 3D (x, y, and z)
    
    # Voxel center estimation    
    dim = (max_xyz_index, 6)
    v_center = np.zeros(dim)                                      # define voxel center array
    v_center[:,0] = np.transpose(np.arange(0,max_xyz_index))      # define column 0 value (voxel index in 3D)
    
    ## Voxel center calculation
    for z in range(0,max_z_index):
        for y in range(0,max_y_index):
            for x in range(0,max_x_index):
                xyidx = (y)*max_x_index + x
                xyzidx = xyidx + (z)*max_xy_index
                
                v_center[xyzidx,1] = p_min[0] - 0.25*v_size + (x-1)*v_size + 0.5*v_size # define column 1 (point center in x axis)
                v_center[xyzidx,2] = p_min[1] - 0.25*v_size + (y-1)*v_size + 0.5*v_size # define column 2 (point center in y axis)
                v_center[xyzidx,3] = p_min[2] - 0.25*v_size + (z-1)*v_size + 0.5*v_size # define column 3 (point center in z axis)
                v_center[xyzidx,4] = xyidx                                              # define column 4 (voxel index in 2D)
                
    # Create voxelled point cloud variable
    dim2 = (max_xyz_index,10)
    voxeled_pt1 = np.zeros(dim2)
    voxeled_pt1[:,0] = v_center[:,0]
    
    voxeled_pt2 = np.zeros(dim2)
    voxeled_pt2[:,0] = v_center[:,0]
    
    # Create input point cloud with voxel ID variable
    dim3_1 = (pt_cloud1[:,0].size,6)
    pt_voxel_ID1 = np.zeros(dim3_1)
    pt_voxel_ID1[:,0] = np.array(np.arange(0,pt_cloud1[:,0].size))
    pt_voxel_ID1[:,1] = pt_cloud1[:,0]
    pt_voxel_ID1[:,2] = pt_cloud1[:,1]
    pt_voxel_ID1[:,3] = pt_cloud1[:,2]
    
    dim3_2 = (pt_cloud2[:,0].size,6)
    pt_voxel_ID2 = np.zeros(dim3_2)
    pt_voxel_ID2[:,0] = np.array(np.arange(0,pt_cloud2[:,0].size))
    pt_voxel_ID2[:,1] = pt_cloud2[:,0]
    pt_voxel_ID2[:,2] = pt_cloud2[:,1]
    pt_voxel_ID2[:,3] = pt_cloud2[:,2]
    
    # Voxel indexing computation
    for i in range(0, pt_cloud1[:,0].size):
        px_idx = math.ceil((pt_cloud1[i,0]-p_min[0]+0.25*v_size)/v_size)
        py_idx = math.ceil((pt_cloud1[i,1]-p_min[1]+0.25*v_size)/v_size)
        pz_idx = math.ceil((pt_cloud1[i,2]-p_min[2]+0.25*v_size)/v_size)
        
        pt_voxel_ID1[i,4] = (py_idx-1)*max_x_index + px_idx                                        # voxel in XY axis
        pt_voxel_ID1[i,5] = pt_voxel_ID1[i,4] + (pz_idx-1)*max_xy_index                            # voxel in XYZ axis
        
        v_center[int(pt_voxel_ID1[i,5]),5] += 1
        
        voxeled_pt1[int(pt_voxel_ID1[i,5]),1] = pt_voxel_ID1[i,0]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),2] = pt_voxel_ID1[i,1]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),3] = pt_voxel_ID1[i,2]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),4] = pt_voxel_ID1[i,3]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),5] = pt_voxel_ID1[i,4]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),6] = intensity1[i]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),7] = reflectance1[i]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),8] = distance1[i]
        voxeled_pt1[int(pt_voxel_ID1[i,5]),9] = np.transpose(angle1)[i]
    for j in range(0, pt_cloud2[:,0].size):
        px_idx = math.ceil((pt_cloud2[j,0]-p_min[0]+0.25*v_size)/v_size)
        py_idx = math.ceil((pt_cloud2[j,1]-p_min[1]+0.25*v_size)/v_size)
        pz_idx = math.ceil((pt_cloud2[j,2]-p_min[2]+0.25*v_size)/v_size)
        
        pt_voxel_ID2[j,4] = (py_idx-1)*max_x_index + px_idx                                        # voxel in XY axis
        pt_voxel_ID2[j,5] = pt_voxel_ID2[j,4] + (pz_idx-1)*max_xy_index                            # voxel in XYZ axis
        
        v_center[int(pt_voxel_ID2[j,5]),5] += 1
        
        voxeled_pt2[int(pt_voxel_ID2[j,5]),1] = pt_voxel_ID2[j,0]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),2] = pt_voxel_ID2[j,1]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),3] = pt_voxel_ID2[j,2]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),4] = pt_voxel_ID2[j,3]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),5] = pt_voxel_ID2[j,4]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),6] = intensity2[j]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),7] = reflectance2[j]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),8] = distance2[j]
        voxeled_pt2[int(pt_voxel_ID2[j,5]),9] = np.transpose(angle2)[j]
    return v_center, voxeled_pt1, voxeled_pt2

In [ ]:
# Import point cloud data
staPoint01, inPoint01, coorPoint01, int_arr01, color01, reflect01, normal01 = data_import(0.000, 0.000, 0.000, "D:/github/data_preparation/ScanPos001_statue_colored_crop2.txt")
staPoint02, inPoint02, coorPoint02, int_arr02, color02, reflect02, normal02 = data_import(-6.357, -2.146, -0.044, "D:/github/data_preparation/ScanPos002_statue_colored_crop2.txt")
staPoint03, inPoint03, coorPoint03, int_arr03, color03, reflect03, normal03 = data_import(-11.921, -12.113, -0.108, "D:/github/data_preparation/ScanPos003_statue_colored_crop2.txt")

In [ ]:
# Angle calculation
angle01, cos_ang01 = np.array(angle_calc(normal01, coorPoint01, staPoint01))
angle02, cos_ang02 = np.array(angle_calc(normal02, coorPoint02, staPoint02))
angle03, cos_ang03 = np.array(angle_calc(normal03, coorPoint03, staPoint03))

In [ ]:
# Distance calculation
dist01 = np.array(euc_distance(staPoint01, coorPoint01)[0])
dist02 = np.array(euc_distance(staPoint02, coorPoint02)[0])
dist03 = np.array(euc_distance(staPoint03, coorPoint03)[0])

In [ ]:
# Voxelization in overlap area
center12, voxel12, voxel21 = voxel_center_overlap(coorPoint01, coorPoint02, int_arr01, int_arr02, reflect01, reflect02, dist01, dist02, angle01, angle02, 0.05)
center13, voxel13, voxel31 = voxel_center_overlap(coorPoint01, coorPoint03, int_arr01, int_arr03, reflect01, reflect03, dist01, dist03, angle01, angle03, 0.05)
center23, voxel23, voxel32 = voxel_center_overlap(coorPoint02, coorPoint03, int_arr02, int_arr03, reflect02, reflect03, dist02, dist03, angle02, angle03, 0.05)

In [ ]:
# Data frame generation
vox12_df = pd.DataFrame(voxel12, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])
vox21_df = pd.DataFrame(voxel21, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])

vox13_df = pd.DataFrame(voxel13, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])
vox31_df = pd.DataFrame(voxel31, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])

vox23_df = pd.DataFrame(voxel23, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])
vox32_df = pd.DataFrame(voxel32, columns= ["3d_idx", "1", "x", "y", "z", "2d_idx", "intensity", "reflectance", "distance", "angle"])

In [ ]:
# Removing no point voxel
vox12_df = vox12_df.loc[vox12_df["distance"] != 0]
vox21_df = vox21_df.loc[vox21_df["distance"] != 0]

vox13_df = vox13_df.loc[vox13_df["distance"] != 0]
vox31_df = vox31_df.loc[vox31_df["distance"] != 0]

vox23_df = vox23_df.loc[vox23_df["distance"] != 0]
vox32_df = vox32_df.loc[vox32_df["distance"] != 0]

In [ ]:
# Array from DataFrame
vox12_df_2 = np.array(vox12_df)
vox21_df_2 = np.array(vox21_df)

vox13_df_2 = np.array(vox13_df)
vox31_df_2 = np.array(vox31_df)

vox23_df_2 = np.array(vox23_df)
vox32_df_2 = np.array(vox32_df)

In [ ]:
# Calculating overlap area
from numba import jit
@jit(nopython=True)
def overlapped_area_each(voxelize01, voxelize02):
    overlapped_area = []
    for i in range(0,len((voxelize01))):
        for j in nb.prange(0,len((voxelize02))):
            if np.transpose(voxelize01)[0][i] == np.transpose(voxelize02)[0][j]:
                overlapped_area_a = voxelize01[i]
                overlapped_area.append(overlapped_area_a)
    return overlapped_area

overlap12 = np.array(overlapped_area_each(vox12_df_2, vox21_df_2))
overlap21 = np.array(overlapped_area_each(vox21_df_2, vox12_df_2))

overlap13 = np.array(overlapped_area_each(vox13_df_2, vox31_df_2))
overlap31 = np.array(overlapped_area_each(vox31_df_2, vox13_df_2))

overlap23 = np.array(overlapped_area_each(vox23_df_2, vox32_df_2))
overlap32 = np.array(overlapped_area_each(vox32_df_2, vox32_df_2))

In [ ]:
def normal_est(coordinate, voxel_center, radius):
    from sklearn.decomposition import PCA
    normal_vector = []
    for i in range(0, len(voxel_center)):
        selecting_point = coordinate[np.where(coordinate[:,0] > (voxel_center[i,1]-radius))]
        selecting_point = selecting_point[np.where(selecting_point[:,0] < (voxel_center[i,1]+radius))]
        selecting_point = selecting_point[np.where(selecting_point[:,1] > (voxel_center[i,2]-radius))]
        selecting_point = selecting_point[np.where(selecting_point[:,1] < (voxel_center[i,2]+radius))]
        selecting_point = selecting_point[np.where(selecting_point[:,2] > (voxel_center[i,3]-radius))]
        selecting_point = selecting_point[np.where(selecting_point[:,2] < (voxel_center[i,3]+radius))]
        
        if len(selecting_point)<3:
            continue
        else:
            pca = PCA(n_components=3)
            pca.fit(selecting_point)
            V = pca.components_.T
            pc3 = np.transpose(V)[0]
            pc3_1 = np.zeros(selecting_point.shape)
            for j in nb.prange(0, len(pc3_1)):
                pc3_1[j] = pc3
            selecting_point = np.append(selecting_point, pc3_1)
            normal_vector.append(pc3)
    normal_vector = np.array(normal_vector)
    return normal_vector

normal_01 = normal_est(coorPoint01, center12, 0.5)
normal_02 = normal_est(coorPoint02, center12, 0.5)
normal_03 = normal_est(coorPoint03, center13, 0.5)